In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
import sys
from sklearn import metrics
from joblib import dump, load
from pathlib import Path

from art.attacks.evasion import FastGradientMethod, ZooAttack
from art.estimators.classification import SklearnClassifier

from art.estimators.classification.scikitlearn import ScikitlearnRandomForestClassifier

from art.metrics import RobustnessVerificationTreeModelsCliqueMethod

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/estimators/certification/__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [3]:
features = ['srcPort', 'destPort',
       'bytes_out', 'num_pkts_out', 'bytes_in', 'num_pkts_in', 'f_ipt_mean',
       'f_ipt_std', 'f_ipt_var', 'f_ipt_skew', 'f_ipt_kurtosis', 'f_b_mean',
       'f_b_std', 'f_b_var', 'f_b_skew', 'f_b_kurtosis', 'duration', 'pr',
       'domainId']
label = 'deviceId'

fieldTypes = {'deviceId': 'int16', 
        #'time_start': 'datetime64[ns]', 'time_end': 'datetime64[ns]', 
        'srcPort': 'int32', 'destPort': 'int32',
        'bytes_out': 'int32', 'num_pkts_out': 'int32',
        'bytes_in': 'int32', 'num_pkts_in': 'int32',
        'f_ipt_mean': 'float64', 'f_ipt_std': 'float64',
        'f_ipt_var': 'float64', 'f_ipt_skew': 'float64',
        'f_ipt_kurtosis': 'float64',
        'f_b_mean': 'float64', 'f_b_std': 'float64',
        'f_b_var': 'float64', 'f_b_skew': 'float64',
        'f_b_kurtosis': 'float64',
        'duration': 'float64', 'pr': 'int8',
        'expire_type': 'object',
        'domain': 'object',
        'domain2': 'object',
        'domainId': 'int16'}

In [4]:
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])
df.fillna(0, inplace=True)

weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]


/var/tmp/pbs.8256194.pbs/ipykernel_451981/2707254373.py:1: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])


In [5]:

#trainCon = df['time'] >= pd.Timestamp('2020-03-01')
#trainCon = df['time'] < pd.Timestamp('2020-01-01')
#trainCon = ((df['time'] >= pd.Timestamp('2020-01-01')) & (df['time'] < pd.Timestamp('2020-03-01')))
trainCon = df['time_start'].dt.isocalendar().week.isin(weeks[2])
#trainCon = df['time'].dt.week.isin([44,45])


#testCon = df['time'] > pd.Timestamp('2020-02-29')
testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
#testCons = pd.date_range('2019-12-01', '2020-05-01', freq = '1W').tolist()
#testCons = pd.date_range('2019-11-01', '2019-12-01', freq = '1W').tolist()

#X = dfTrain.iloc[:, 2:5]
#y = dfTrain.iloc[:, 1]
X = df[trainCon][features]
y = df[trainCon][label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [19]:
rfc = RandomForestClassifier(n_jobs=18)
modelFile = Path(f"model/rfc_justvalues_0")
if modelFile.is_file():
    print(f"loading model {modelFile}")
    rfc = load(modelFile)

loading model model/rfc_justvalues_0


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    
    #y_pred = clf.predict(X)
    y_rfc = rfc.predict(X)
    #print("rfc predict")
    #y_svc = svc.predict(X)
    #print("svc predict")
    #y_dtc = dtc.predict(X)
    #print("dtc predict")
    #y_knn = knn.predict(X)
    #print("knn predict")
    #y_mv = mv.predict(X)
    #print("mv predict")
    
    print("Benign test")
    print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))
    print(classification_report(y,y_rfc))#, labels=devs))
            #metrics.f1_score(y, y_svc, average='micro'),
            #metrics.f1_score(y, y_dtc, average='micro'),
            #metrics.f1_score(y, y_knn, average='micro'),
            #metrics.f1_score(y, y_mv, average='micro')
            #)
    #print(f"Accuracy {date.date}:", metrics.accuracy_score(y, y_pred))

    # Step 6: Generate adversarial test examples
    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Benign test
44 0.6970325039829663


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.60      0.99      0.75      2543
           3       0.56      0.94      0.70      6433
           4       0.00      0.00      0.00         0
           5       1.00      0.04      0.08       268
           6       0.36      0.21      0.26     15270
           7       0.92      0.82      0.87      2047
           8       0.00      0.00      0.00      5563
           9       0.94      0.74      0.83      1945
          10       0.46      0.96      0.62     11969
          11       0.62      0.24      0.34      2231
          12       0.82      0.46      0.59      2882
          13       0.91      0.90      0.90     43935
          14       0.32      0.93      0.48      3266
          15       0.65      0.38      0.48      6868
          16       0.42      0.35      0.38     18239
          17       0.00      0.00      0.00         0
          18       0.82    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


...

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Benign test
17 0.980216300374232


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.97      0.96      0.97     14400
           2       0.98      0.97      0.97     15850
           3       1.00      1.00      1.00     26618
           4       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.87      0.94      0.90      4554
           9       0.95      0.94      0.95     14376
          10       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.99      1.00      1.00    278553
          14       0.97      0.98      0.98     16755
          15       0.95      0.91      0.93      7782
          16       0.00      0.00      0.00         0
          17       0.99      0.99      0.99     21345
          18       0.95      0.95      0.95     21538
          19       0.97      0.97      0.97     33089
          20       0.94      0.91      0.92      8756
          23       0.95    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
advClassifier = SklearnClassifier(rfc)
zoo = ZooAttack(classifier=advClassifier, confidence=0.0, targeted=False, learning_rate=0.1, max_iter=5,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
dist = 0
for date in testCons:
    path = Path(f"RFzoo2TARGETED-{date.week}.csv")
    if path.is_file():
        print("test exists")
        continue
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
    
    if len(X) == 0: 
        continue
    x_test_adv = zoo.generate(x=X_test.values, y=y_test-2)
   
    np.savetxt(f"RFzoo2TARGETED-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    y_rfc = advClassifier.predict(x_test_adv)
    for index, sample in enumerate (x_test_adv):
        dist = dist + np.linalg.norm(sample-X_test[index])
    print(dist)
    #print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))
    
    


ZOO: 100%|██████████| 200/200 [03:56<00:00,  1.18s/it]


KeyError: 0

In [21]:
dist = 0
for date in testCons:
    path = Path(f"RFzoo-{date.week}.csv")
    if path.is_file():
        print("test exists")
        x_test_adv = np.loadtxt(f"RFzoo-{date.week}.csv", delimiter=",", dtype=str)
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
    
    if len(X) == 0: 
        continue
    #x_test_adv = zoo.generate(x=X_test.values, y=y_test-2)
   
    #np.savetxt(f"RFzoo2-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    #y_rfc = advClassifier.predict(x_test_adv)
    #for index, sample in enumerate (x_test_adv):
    #    dist = dist + np.linalg.norm(sample-X_test[index])
    #print(dist)
    print(rfc.score(x_test_adv, y_test))

test exists
0.07
test exists
0.075
test exists
0.07
test exists
0.08
test exists
0.065
test exists
0.105
test exists
0.125
test exists
0.085
test exists
0.065
test exists
0.09
test exists
0.12
test exists
0.08
test exists
0.07
test exists
0.085
test exists
0.105
test exists
0.175
test exists
0.075
test exists
0.06
test exists
0.105
test exists
0.125
test exists
0.135
test exists
0.14
test exists
0.115
test exists
0.17
test exists
0.2
test exists
0.155


In [1]:
dist=0
for index, sample in enumerate (x_test_adv):
    dist = dist + np.linalg.norm(sample-X_test.values[index])
print(dist)


NameError: name 'x_test_adv' is not defined

In [25]:
from art.metrics import RobustnessVerificationTreeModelsCliqueMethod
aclassifier = SklearnClassifier(model=rfc)
rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=aclassifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


KeyboardInterrupt: 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y)
X_test = normalize(X=X_test)

rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=advClassifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


KeyboardInterrupt: 

In [24]:
NB_TRAIN = 100
NB_TEST = 100

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

n_classes = 41
n_features = 28 * 28
n_train = X_train.shape[0]
n_test = X_test.shape[0]
#X_train = X_train.reshape((n_train, n_features))
#X_test = X_test.reshape((n_test, n_features))

X_train = X_train[:NB_TRAIN]
y_train = y_train[:NB_TRAIN]
X_test = X_test[:NB_TEST]
y_test = y_test[:NB_TEST]

In [1]:
model = load(modelFile)
classifier = SklearnClassifier(model=model)

rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=classifier)
average_bound, verified_error = rt.verify(x=x_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

NameError: name 'load' is not defined